In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain.llms import HuggingFacePipeline
from langchain import hub
from langchain.chains.retrieval import create_retrieval_chain



import os
import transformers
import torch
import tqdm as notebook_tqdm

c:\Users\email\Desktop\2504_Aging_news\2504_Aging_news\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()
hf_token = os.getenv('HF_TOKEN')

In [ ]:
# Load the Llama 3 model and tokenizer
# running time was 7 min
model_id =  "meta-llama/Llama-3.2-1B"#"meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_id, token=hf_token)

In [4]:
text_generation_pipeline = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

Device set to use cpu


In [ ]:
# Create a text generation pipeline (it was without tranformers)
text_generation_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256, #max_length=512, #was 1024
    temperature=0.5, # it was 0.7
    top_p=0.8, # it was 0.9
    repetition_penalty=1.1
)



In [5]:
# Wrap the pipeline in a LangChain LLM
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

C:\Users\email\AppData\Local\Temp\ipykernel_3804\751951455.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [6]:
def retrieve_from_vector_db(vector_db_path):
    # FAISS.load_local needs the embeddings object to load the vector store
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2",
    )
    doc_vectorStore = FAISS.load_local(
        folder_path=vector_db_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True,
    )
    retriever = doc_vectorStore.as_retriever()
    return retriever

In [7]:
def connect_chains(retriever):
    # This chain takes a list of documents and formats them all into a prompt, then passes that prompt to the LLM. It passes all documents, so we need to make sure it fits within the context window of the LLM.
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat"),
    ) 
    # this chain takes in a user inquiry, which is then passed to the retriever to fetch relevant documents. Those documents (and original inputs) are then passed to the LLM to generate response. 
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain

In [8]:
docName = "docum"

In [9]:
custom_retriever = retrieve_from_vector_db("../data/vector_databases/"+docName+"_vector_db")

C:\Users\email\AppData\Local\Temp\ipykernel_3804\1803034753.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [10]:
retrievalChain = connect_chains(custom_retriever)

c:\Users\email\Desktop\2504_Aging_news\2504_Aging_news\.venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
user_dialog = "Please write me a summary of the text"

In [16]:
def print_output(inquiry, retrieval_chain=retrievalChain):
    result = retrieval_chain.invoke({"input": inquiry})
    print(result['answer'].strip("\n"))

In [15]:
print_output(user_dialog)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


System: Answer any use questions based solely on the context below:

<context>
56:20 
That's a big deal. This is, I think, a turning point in medical history, that we actually understand 
56:26 
what causes those diseases fundamentally, not just trying to fix the actual problem when it occurs. 
56:32 
I liken this to get getting to the edge of a cliff. And you wrote this in the book, 
56:38 
is that we've been far too often working on trying to understand why we falling off a cliff 
56:44 
at the end of life, without even asking the question, "What brings us to the edge of that cliff in the first 
place." Now we finally understand how we get to that cliff, 
56:51 
and even how to prevent it. - And so at some point perhaps we don't have to worry about diseases 
56:59 
because we're going to be so healthy. So young that they're not going to push us off. 
57:06 
They're going to make us sick, maybe. We can treat that. But- - Well, as we saw with COVID-19, that if
you're young, 
57:12

of 

'System: Answer any use questions based solely on the context below:\n\n<context>\n56:20 \nThat\'s a big deal. This is, I think, a turning point in medical history, that we actually understand \n56:26 \nwhat causes those diseases fundamentally, not just trying to fix the actual problem when it occurs. \n56:32 \nI liken this to get getting to the edge of a cliff. And you wrote this in the book, \n56:38 \nis that we\'ve been far too often working on trying to understand why we falling off a cliff \n56:44 \nat the end of life, without even asking the question, "What brings us to the edge of that cliff in the first \nplace." Now we finally understand how we get to that cliff, \n56:51 \nand even how to prevent it. - And so at some point perhaps we don\'t have to worry about diseases \n56:59 \nbecause we\'re going to be so healthy. So young that they\'re not going to push us off. \n57:06 \nThey\'re going to make us sick, maybe. We can treat that. But- - Well, as we saw with COVID-19, that if

In [17]:
print_output("What is the main idea of the text?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


System: Answer any use questions based solely on the context below:

<context>
ourselves and others, 
3:20 
is that we are at a turning point in medical history. We finally, as a species, 
3:25 
understand how to control our biology, how to optimize our bodies, during early life, mid-life and 
certainly late life, 
3:33 
to lead lives that can be decades longer and healthier. And we want to share that information. 
3:38 
It's very hard for the average person to understand, let alone digest, thousands of scientific papers. 
3:45 
That's what I do. I have a team of researchers and in fact, today, we're supported by a team of 
researchers who have helped us gather information

of where we were in Lifespan. 
12:21 
There's a lot to talk about, including things that you can apply in your daily lives, that we didn't know 
when Lifespan was written. 
12:27 
- One of misconceptions I think a lot of people have about the book, a lot of people have about your 
research, is they think that you're